In [2]:
### load the functions
load('Zsap.sage')
### find these files on 
### https://github.com/jephianlin/publish

In [ ]:
### This file is copied from https://sage.math.iastate.edu/home/pub/54/
### which was cited by
### J. C.-H. Lin. Using a new zero forcing process to guarantee the Strong Arnold Property. Linear Algebra Appl., 507:229--250, 2016.

### Moved to Github on January, 2020.

In [ ]:
#Code for the data in the Table of all parameters.
#Computing the number of graphs with beta=0, where beta is one type of Zsap variations.

print("n: number of vertices")
print("a0: number of connected graphs on n vertices")
print("a1: number of connected graphs on n vertices with Zsap=0")
print("a2: number of connected graphs on n vertices with Zsap^ell=0")
print("a3: number of connected graphs on n vertices with Zsap^plus=0")
print("---")
print("n [a1/a0,a2/a0,a3/a0]")
print("  [sap, ell,   +]")
data={}
proportion={};
for n in range(1,11):
    all_counter=0;
    Zsap_counter=0;
    Zsapell_counter=0;
    Zsapplus_counter=0;
    for g in graphs.nauty_geng("%s -c"%n):
        all_counter+=1;
        if find_Zsap(g,oc_rule=True):
            Zsap_counter+=1;
        if find_Zsap(g,"CCRZell",oc_rule=True):
            Zsapell_counter+=1;
        if find_Zsap(g,"CCRZplus",oc_rule=True):
            Zsapplus_counter+=1;
    data[n]=[all_counter,Zsap_counter,Zsapell_counter,Zsapplus_counter];
    all=data[n][0]
    proportion[n]=[];
    for i in range(1,4):
        proportion[n].append(N(data[n][i]/all,digits=2))
    print("%s %s"%(n,proportion[n]));
print("---");
print("n [a0,a1,a2,a3]");
for n in range(1,11):
    print("%s %s"%(n,data[n]));

### Expected outcome: ###
#n: number of vertices
#a0: number of connected graphs on n vertices
#a1: number of connected graphs on n vertices with Zsap=0
#a2: number of connected graphs on n vertices with Zsap^ell=0
#a3: number of connected graphs on n vertices with Zsap^plus=0
#---
#n [a1/a0,a2/a0,a3/a0]
#  [sap, ell,   +]
#1 [1.0, 1.0, 1.0]
#2 [1.0, 1.0, 1.0]
#3 [1.0, 1.0, 1.0]
#4 [1.0, 1.0, 1.0]
#5 [0.86, 0.95, 0.95]
#6 [0.79, 0.92, 0.92]
#7 [0.74, 0.89, 0.89]
#8 [0.73, 0.88, 0.88]
#9 [0.76, 0.89, 0.89]
#10 [0.79, 0.90, 0.91]
#---
#n [a0,a1,a2,a3]
#1 [1, 1, 1, 1]
#2 [1, 1, 1, 1]
#3 [2, 2, 2, 2]
#4 [6, 6, 6, 6]
#5 [21, 18, 20, 20]
#6 [112, 88, 103, 103]
#7 [853, 628, 756, 757]
#8 [11117, 8164, 9753, 9784]
#9 [261080, 197895, 231541, 232580]
#10 [11716571, 9242060, 10576775, 10636311] 

In [ ]:
###Back up for data and proportion
databk={1: [1, 1, 1, 1], 2: [1, 1, 1, 1], 3: [2, 2, 2, 2], 4: [6, 6, 6, 6], 5:
[21, 18, 20, 20], 6: [112, 88, 103, 103], 7: [853, 628, 756, 757], 8:
[11117, 8164, 9753, 9784], 9: [261080, 197895, 231541, 232580], 10:
[11716571, 9242060, 10576775, 10636311]};

proportionbk={1: [1.0, 1.0, 1.0], 2: [1.0, 1.0, 1.0], 3: [1.0, 1.0, 1.0], 4: [1.0,
1.0, 1.0], 5: [0.86, 0.95, 0.95], 6: [0.79, 0.92, 0.92], 7: [0.74, 0.89,
0.89], 8: [0.73, 0.88, 0.88], 9: [0.76, 0.89, 0.89], 10: [0.79, 0.90,
0.91]} 

In [ ]:
###Build function to confirm xi(G)=ZFloor(G) for |G|<=7.
###Miscellaneous functions: get_Z, find_ZFloor, has_minor, NoT3.

def get_Z(g):
    """
    Input:
        g: a simple graph
    Output:
        return the value of the conventional zero forcing number Z(g).
    """
    V=g.vertices();
    n=g.order();
    lbd=-1;
    ubd=n;
    while ubd-lbd>=2: #apply random algorithm;
        guess=random.choice(range(lbd+1,ubd)); #take an interior point in [lbd,ubd];
        found=False;
        for sub in Combinations(V,guess):
            if n==len(Z_game(g,sub)):
                   found=True;
                   break;
        if found:
            ubd=guess;
        else:
            lbd=guess;
    return ubd;

def ZFloor_game(g,done,act,token):
    """
    Input:
        g: a simple graph
        done: list of blue vertices that made their force (they shouldn't have white neighbors)
        act: list of blue vertices that make no force yet
        token: number of "free" blue vertices that can make a hop
    Output:
        if "act" together with "token" number of free blue vertices is a zero forcing set,
         then return True. Otherwise, return False.
        **ZFloor_game(g,[],[],k) is returning if ZFloor(g)<=k or not.
    """
    #recursive algorithm is implemented, so each graph and each list should be copied
    h=g.copy() 
    this_done=[];
    this_act=[];
    for v in done:
        h.delete_vertex(v);
        this_done.append(v);
    for v in act:
        this_act.append(v);        
    #delete every edges between this_act.
    for u,w in Combinations(this_act,2):
        h.delete_edge(u,w);
    #Do one round of propagation according to CCR-Z
    again=True;
    while again:
        again=False;
        for v in this_act:
            if h.degree(v)==1: #this vertex can make CCR-Z. After that it is done, and cannot make a hope.
                u=h.neighbors(v)[0];
                this_act.append(u);
                this_act.remove(v);
                this_done.append(v);
                h.delete_vertex(v);
                for w in this_act:
                    h.delete_edge(u,w);
                again=True;
                break;                    
            if h.degree(v)==0: #this vertex can make a hop. token+1, delete it from h, and add it to done.
                token+=1;
                this_act.remove(v);
                this_done.append(v);
                h.delete_vertex(v);
                again=True;
    #When the while loop is done, all token are collected, and all vertices in act cannot make a force.
    if h.order()==0:
        return True;
    if h.order()!=0 and token==0:
        return False;
    #Find white set. And do recursion.
    white=h.vertices();
    for v in this_act:
        white.remove(v);
    if token>=len(white):
        return True;
    else:
        for new_act in Combinations(white,token): #try all possible way to put the tokens
            if ZFloor_game(g,this_done,this_act+new_act,0)==True:
                return True;
        return False;
        
def find_ZFloor(g):
    """
    Input:
        g: a simple graph
    Output:
        return the value of ZFloor(g).
    """
    ZF=g.order()-1;
    if ZF<0: #define ZFloor(null graph)=0
        return ZF+1;
    try_lower=True;
    while try_lower:
        try_lower=False;
        if ZFloor_game(g,[],[],ZF)==True:
            try_lower=True;
            ZF+=-1;     
    return ZF+1;

###T3Family, the forbidden graphs for xi(G)<=2.
T3FamilyString=['C~', 'DFw','EBnW','F@QZo','G?Gisg','H??@qiK']
T3Family=[Graph(stg) for stg in T3FamilyString];

## The code for has_minor comes from http://ask.sagemath.org/question/
## 8112/graph-minor-code-too-slow-in-certain-situations-sage-46/
def has_minor(G, H):
    #return if G has a H minor or not.
    try:
        m = G.minor(H)
        return True
    except ValueError: 
        return False

def NoT3(g):
    #return True if g has a T3 minor. This means xi(G)>=3.
    #otherwise, return False. This means xi(G)<=2.
    ###usually SUPER slow...
    for t in T3Family:
        if has_minor(g,t):
            return True;
    return False; 
       	
###According to the order (time complexity), check if one of the following cases applies:
### - Zsapoc(G)=0;
### - G is a tree;
### - ZFloor(G)=M(G)-Zvcoc(G);
### - ZFloor(G)=Hadwiger number-1;
### - ZFloor(G)=3 and G has no T3 family;

print("n: number of vertices");
print("all: number of connected graphs on n vertices");
print("categorized: number of graphs that fall into at least one of the categories");
print("ai=number of graphs that fall into the i-th category but not any of the previous categories");
print("---");
print("n","[all, categorized]","categorized/all","[a1,a2,a3,a4,a5]");
remainder=[]
for n in range(1,8):
    counter=0;
    Zsap_counter=0;
    extra_trees=0;
    extra_Zvcoc=0;
    extra_Knminor=0;
    extra_T3=0;
    for g in graphs.nauty_geng("%s -c"%n):
        counter+=1;
        if find_Zsap(g,oc_rule=True):
            Zsap_counter+=1;
        elif g.is_tree():
            extra_trees+=1;
        else:
            zfloor=find_ZFloor(g);
            M=get_Z(g); #Up to seven vertices, M(G)=Z(G)
            zvcoc=find_Zsap(g,oc_rule=True,get_value="vertex");
            if zfloor==M-zvcoc:
                extra_Zvcoc+=1;
            elif has_minor(g,graphs.CompleteGraph(zfloor+1)):
                extra_Knminor+=1;
            elif zfloor==3 and NoT3(g)==True:
                extra_T3+=1;
            else:
                remainder.append(g.canonical_label().graph6_string());
                sshow(g);
    sum=Zsap_counter+extra_trees+extra_Zvcoc+extra_Knminor+extra_T3;
    print("%s %s %s %s"%(n,[counter,sum],N(sum/counter,digits=2),[Zsap_counter,extra_trees,extra_Zvcoc,extra_Knminor,extra_T3]));
print("Graphs that doesn't fall into any of the five categories %s"%remainder);

### Expected outcome: ###
#n: number of vertices
#all: number of connected graphs on n vertices
#categorized: number of graphs that fall into at least one of the
#categories
#ai=number of graphs that fall into the i-th category but not any of the
#previous categories
#---
#n [all, categorized] categorized/all [a1,a2,a3,a4,a5]
#1 [1, 1] 1.0 [1, 0, 0, 0, 0]
#2 [1, 1] 1.0 [1, 0, 0, 0, 0]
#3 [2, 2] 1.0 [2, 0, 0, 0, 0]
#4 [6, 6] 1.0 [6, 0, 0, 0, 0]
#5 [21, 21] 1.0 [18, 1, 1, 1, 0]
#6 [112, 112] 1.0 [88, 3, 12, 9, 0]
#7 [853, 853] 1.0 [628, 7, 125, 78, 15]
#Graphs that doesn't fall into any of the five categories [] 